<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Pesquisa/4_Teste_Massivo_Modularizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [48]:
import os

# A biblioteca "os" fornece funções para interagir com o sistema operacional, como manipulação de caminhos de arquivo, 
# verificação de existência de arquivos/diretórios, criação de diretórios, etc.

import glob

# A biblioteca "glob" é usada para obter uma lista de caminhos de arquivos que correspondem a um determinado padrão. 
# É útil para buscar arquivos em um diretório com base em padrões de nomenclatura ou extensões.

import cv2

# A biblioteca "OpenCV" (Open Source Computer Vision Library) é uma biblioteca popular para processamento de imagem e visão computacional. 
# Ela fornece várias funções e algoritmos para manipulação, análise e processamento de imagens e vídeos.

import numpy as np

# A biblioteca "NumPy" é usada para operações numéricas eficientes em Python. 
# Ela fornece arrays multidimensionais (ndarrays) que são usados para armazenar e manipular dados numéricos.

import pandas as pd

# A biblioteca "Pandas" fornece estruturas de dados e funções para análise de dados. 
# Ela é amplamente utilizada para manipulação e análise de dados tabulares, como planilhas ou bancos de dados.

from google.colab.patches import cv2_imshow

from google.colab import drive

# A biblioteca "drive" permite montar o Google Drive no ambiente do Google Colab, 
# facilitando o acesso a arquivos e diretórios armazenados no Google Drive.

drive.mount('/content/drive/')

# Monta o Google Drive no ambiente do Google Colab, permitindo acessar os arquivos armazenados no Google Drive.

%cd /content/drive/MyDrive/

# Altera o diretório de trabalho atual para o diretório especificado no Google Drive.

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


# Funções:

## Create_Dataset:

In [49]:
def create_dataset(dataset):
  # Cria uma lista vazia para armazenar os objetos
  objects = []

  # Percorre o dicionário
  for attr, obj_list in dataset.items():
    # Percorre a lista de objetos para o atributo atual
    for obj in obj_list:
      # Verifica se obj é um dicionário
      if isinstance(obj, dict):
        # Adiciona o objeto combinando com o dicionário {'attribute': attr}
        objects.append({**{'attribute': attr}, **obj})

  # Cria o DataFrame com base na lista de objetos
  df = pd.DataFrame(objects)

  # Retorna o DataFrame se for necessário usá-lo posteriormente
  return df

## calculate_contour_properties:

In [50]:
def calculate_contour_properties(cnt):
  # Momentos do objeto
  M = cv2.moments(cnt)

  # Calcula a área do contorno
  area = cv2.contourArea(cnt)

  # Calcula o perímetro do contorno
  perimeter = cv2.arcLength(cnt, True)

  # Proporção de aspecto do objeto
  x,y,w,h = cv2.boundingRect(cnt)
  width = w
  height = h

  if min(w, h) == 0:
    aspect_ratio = 0
  else:
    aspect_ratio = max(float(w), h) / min(float(w), h)

  # Ângulo e raio mínimo envolvente em torno do objeto
  rect = cv2.minAreaRect(cnt)
  circle = cv2.minEnclosingCircle(cnt)

  box = cv2.boxPoints(rect)
  box = np.int0(box)

  # Extensão do objeto (razão entre a área do objeto e a área do retângulo envolvente mínimo)
  rect_area = w*h

  # Forma de achar o contorno 1
  center_x = int(M["m10"] / M["m00"])
  center_y = int(M["m01"] / M["m00"])

  # Forma de achar o contorno 2 - Circulo
  (x_circle, y_circle), radius = cv2.minEnclosingCircle(cnt)
  center_circle = (int(x_circle), int(y_circle))
  cx_circle = int(x_circle)
  cy_circle = int(y_circle)

  # Forma de achar o contorno 2 - Quadrado
  cx_quad = int(x)
  cy_quad = int(y)

  
  # Calcula a circularidade do contorno
  if perimeter > 0:
      circularity = 4 * np.pi * area / (perimeter ** 2)
  else:
      circularity = 0
      
  return area, circularity, radius, x, y, w, h, aspect_ratio, box, center_circle, cx_circle, cy_circle, cx_quad, cy_quad, center_x, center_y

## process_contours:

In [51]:
def process_contours(nome_frame, img_contours, contours, imagem_original):

  objetos = 0
  dicionario = {}  # Dicionário para armazenar os dados dos objetos identificados

  for cnt in contours:
    # Aproxima o contorno por uma sequência de segmentos de linha
    approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)

    # Calcula as propriedades do contorno
    area, circularity, radius, x, y, w, h, aspect_ratio, box, center, cx_circle, cy_circle, cx_quad, cy_quad, center_x, center_y = calculate_contour_properties(cnt)

    # Para um padrão Circular:
    if circularity > 0.30 and area > 2000 and radius < 100 and radius > 20:

      # Define o raio máximo desejado
      max_radius = 100

      # Limita o raio ao valor máximo
      radius = min(radius, max_radius)

      # Converte o raio para inteiro
      radius = int(radius)

      # Desenha um contorno Circular
      cv2.circle(img_contours, center, radius, (255, 0, 0), 2)
      
      # Desenho do ponto central
      cv2.circle(img_contours, (center_x, center_y), 5, (255, 0, 0), -1) # Azul - Momentun
      cv2.circle(img_contours, (cx_circle, cy_circle), 5, (0, 255, 0), -1) # Verde - minEnclosingCircle

      # Contagem total de Objetos Identificados
      objetos = objetos + 1

      # Captura dos dados:
      dicionario[f"{nome_frame}_{objetos}"] = []
      # Adicione os dados desejados ao dicionário dataset
      dicionario[f"{nome_frame}_{objetos}"].append({
          "area": area,
          "circularity": circularity,
          "radius": radius,
          "Form": "circle",
          "x": x,
          "y": y,
          "w": w,
          "h": h,
          "aspect_ratio": aspect_ratio,
          "box": box,
          "center": center,
          "cx_circle": cx_circle,
          "cy_circle": cy_circle,
          "cx_momentun": center_x,
          "cy_momentun": center_y,
          #"ponto_ideal_x": ,
          #"ponto_ideal_y": ,
      })


    else:
      if aspect_ratio < 2 and area > 2000 and area < 6543.5: 
        # Desenha um contorno quadratico
        cv2.drawContours(img_contours, [box], 0, (0, 0, 255), 2)

        # Desenho do ponto central
        cv2.circle(img_contours, (center_x, center_y), 5, (0, 0, 255), -1)
        cv2.circle(img_contours, (cx_quad, cy_quad), 5, (0, 255, 0), -1)

        # Contagem total de Objetos Identificados
        objetos = objetos + 1
        dicionario[f"{nome_frame}_{objetos}"] = []
        dicionario[f"{nome_frame}_{objetos}"].append({
            "area": area,
            "circularity": circularity,
            "radius": radius,
            "Form": "square",
            "x": x,
            "y": y,
            "w": w,
            "h": h,
            "aspect_ratio": aspect_ratio,
            "box": box,
            "center": center,
            "cx_circle": cx_circle,
            "cy_circle": cy_circle,
            "cx_quad": cx_quad,
            "cy_quad": cy_quad,
            "center_x": center_x,
            "center_y": center_y
            #"ponto_ideal_x": ,
            #"ponto_ideal_y": ,
        })

  return imagem_original, objetos, dicionario

## find_contours:

In [52]:
def find_contours(image):
  # Encontra os contornos na imagem
  contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  return contours

## dilate_image:

In [53]:
def dilate_image(image):
  # Define o kernel para a dilatação
  kernel = np.ones((5, 5), np.uint8)

  # Dilata a imagem
  image = cv2.dilate(image, kernel, iterations=2)
  return image

## apply_threshold:

In [54]:
def apply_threshold(image):
  # Aplica um filtro de Sobel em x e y
  sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
  sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

  # Calcula a magnitude do gradiente
  gradient_magnitude = cv2.magnitude(sobelx, sobely)

  # Normaliza a magnitude do gradiente
  gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

  # Aplica uma limiarização utilizando o método de Otsu
  _, gradient_threshold = cv2.threshold(gradient_magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  
  return gradient_threshold

## apply_filters:

In [55]:
def apply_filters(image):
  # Converte a imagem para float32
  image32f = np.float32(image)
  
  # Aplica um filtro de média
  mu = cv2.blur(image32f, (3, 3))

  # Calcula a variância da imagem filtrada
  mu2 = cv2.blur(image32f * image32f, (3, 3))
  sigma = cv2.sqrt(mu2 - mu * mu)
  sigma = sigma * 10
  sigma = sigma.astype("uint8")

  # Aplica um filtro de borramento Gaussiano
  sigma = cv2.GaussianBlur(sigma, (5, 5), 0)

  # Aplica o filtro Laplaciano
  laplacian = cv2.Laplacian(sigma, cv2.CV_32F)

  # Converte a saída para o intervalo de 0 a 255
  sigma = cv2.convertScaleAbs(laplacian)

  return sigma

## print_frame_info:

In [56]:
def print_frame_info(nome):
  # Imprime informações do frame
  print(f"================================================================================")
  print(f"================================================================================")
  print(f"Amostra: " + nome)

## process_frame:

In [57]:
def process_frame(frame):
  # Carrega a imagem
  im = cv2.imread(frame)
  
  # Divide o caminho do arquivo e obtém o nome do frame
  nome_completo = frame.split('/')
  nome = nome_completo[int(posicao_do_nome_frame)].split('.')
  amostra = nome[0]

  # Imprime informações do frame
  print_frame_info(amostra)

  # Cria uma cópia da imagem para desenhar os contornos
  img_contours = im.copy()

  # Converte a imagem para escala de cinza
  image = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  # Aplica uma série de filtros à imagem
  sigma = apply_filters(image)

  # Aplica uma limiarização à imagem filtrada
  gradient_threshold = apply_threshold(sigma)

  # Dilata a imagem limiarizada
  gradient_threshold = dilate_image(gradient_threshold)

  # Encontra os contornos na imagem dilatada
  contours = find_contours(gradient_threshold)

  # Processa cada contorno encontrado
  resultados = process_contours(amostra, img_contours, contours, img_contours)

  return resultados, amostra

## remove_rats

In [58]:
# Função para remover os ratos e obter a imagem resultante
def remove_rats(images):

  # Convertendo as imagens em formato de array numpy individualmente
  images_array = [np.array(image) for image in images]

  # Calculando a mediana ao longo do eixo temporal (eixo 0)
  median_image = np.median(images_array, axis=0)

  # Convertendo a imagem resultante de volta para o tipo de dados uint8
  median_image = median_image.astype('uint8')

  return median_image

## extract_frame_from_video:

In [59]:
def extract_frames_from_videos(entrada, destino, cont=1):

  # Lista para armazenar o número "x"
  images = []

  cont = int(cont)

  for video in entrada:

    # Verificar se o caminho do vídeo existe
    if not os.path.exists(video):
      print(f"O vídeo '{video}' não existe.")
      return

    # Abrir o vídeo
    vidcap = cv2.VideoCapture(video)

    # Verificar se o vídeo foi aberto corretamente
    if not vidcap.isOpened():
      print(f"Falha ao abrir o vídeo '{video}'.")
      return

    success, image = vidcap.read()

    if not success:
      # break out of the loop if there are no frames to read
      break

    interval = 60  # intervalo em segundos (1 minuto)
    count = 0

    # Divide o caminho do arquivo e obtém o nome do frame
    nome_completo = video.split('/')
    nome = nome_completo[9].split('.')

    # start the loop
    while success and count < cont:
      # save frame as JPEG file
      success, image = vidcap.read()

      if image is None:
        # No more frames to read, break out of the loop
        break

      clone = image.copy()

      images.append(image)  # Criar uma cópia da imagem para evitar referências indesejadas

      # increment the frame count
      count += 1

      # skip frames based on the interval
      vidcap.set(cv2.CAP_PROP_POS_FRAMES, int(vidcap.get(cv2.CAP_PROP_POS_FRAMES)) + interval * vidcap.get(cv2.CAP_PROP_FPS))

    # Chamar a função remove_rats com a lista de números "x"
    result = remove_rats(images)

    output_path = os.path.join(destino, f"{nome[0]}.jpg")
    print(output_path)

    # Salvar a imagem resultante
    cv2.imwrite(output_path, result)

    # Limpar a lista de imagens para o próximo vídeo
    images.clear()

# Base do Sistema:

## Endereço dos Videos:

In [60]:
# entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia II - Dados/Experimentos comportamento II/Experimento 12/videos_separados/TR e TT/grey_field/female/*.mp4')

# entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 15/Vídeos separados - TR e TT/grey_field_videos/*.mp4')

entrada = glob.glob('/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/*.mpg')

# Imprime os elementos da lista com quebras de linha entre cada elemento
print('\n'.join(entrada))

/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF504-Contexto4.mpg
/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF508-Contexto4.mpg
/content/drive/Shareddrives/Lab Electrofisiologia III - Dados/Experimentos comportamento III/Experimento 17/Vídeos separados (explo)/campo preto/Dia2-Explo-CCF512-Contexto4.mpg


In [61]:
posicao_do_nome_video = input("Posição do nome do video: ") #9

Posição do nome do video: 9


In [62]:
cont = input("Quantas imagens serão usadas como base pro processo: ") #5

Quantas imagens serão usadas como base pro processo: 5


## Endereço das imagens Brutas:

In [63]:
#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Base de Dados/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_grey/'

destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Imagens_Infravermelho/'

#destino = '/content/drive/MyDrive/Pesquisa_ISD_IC/Teste_Massivo/'

## Extraindo os Frames dos Videos:

In [64]:
#extract_frames_from_videos(entrada, destino, cont)

### Listando Imagens na pasta destino:

In [ ]:
frames = glob.glob(destino + '*.jpg')
print('\n'.join(frames))

In [66]:
posicao_do_nome_frame = input("posicao_do_nome_frame:") #6

posicao_do_nome_frame:6


# Algoritmo Principal:

In [ ]:
# Loop que processa cada arquivo na lista de caminhos de arquivos.
for frame in frames:
  resultados, amostra = process_frame(frame)

  # Imprime o número de objetos encontrados
  print(resultados[1])

  # Exibe a imagem com os contornos encontrados
  cv2_imshow(resultados[0])

  output_path = os.path.join(destino, f"{amostra}.jpg")

  # Salvar a imagem resultante
  cv2.imwrite(output_path, resultados[0])

  # Chama a função create_dataset com o dataset gerado
  dados = create_dataset(resultados[2])

  dados.head()

  # Salva o DataFrame em um arquivo CSV
  dados.to_csv(output_path + "dataset", index=False)



In [68]:
dados.head()

,attribute,area,circularity,radius,Form,x,y,w,h,aspect_ratio,box,center,cx_circle,cy_circle,cx_quad,cy_quad,center_x,center_y
0,Dia5-TT-CCF512-Contexto4_frame4_1,2124.5,0.695074,32,circle,195,252,62,50,1.240000,"[[184, 290], [204, 244], [258, 268], [239, 313]]","(225, 276)",225,276,195,252,221,276
1,Dia5-TT-CCF512-Contexto4_frame4_2,6385.5,0.655326,51,circle,365,220,87,98,1.126437,"[[362, 222], [449, 218], [453, 314], [366, 318]]","(403, 269)",403,269,365,220,407,271
2,Dia5-TT-CCF512-Contexto4_frame4_3,9283.0,0.598131,80,circle,395,5,152,89,1.707865,"[[394, 92], [395, 4], [546, 6], [545, 93]]","(468, 43)",468,43,395,5,482,44


In [69]:
len(resultados[0])

480